# Data Cube View transmission efficiency ration uniformity to produce multi gif

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- creation date : February 18th 2022 

## Import

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import scipy.stats
import sys
sys.path.append('../')

In [ ]:
plt.rcParams["figure.figsize"] = (16,4)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams['figure.max_open_warning'] = 0

In [ ]:
import os

In [ ]:
print(sys.executable)
print(sys.version)
#print(sys.version_info)

In [ ]:
from scipy.interpolate import griddata
from scipy import interpolate
from scipy.interpolate import Rbf

## Configuration

In [ ]:
WL = np.arange(441,1020,2) # for interpolation cannot do less
NWL = len(WL)

In [ ]:
NWL

In [ ]:
XMIN = -4.5
XMAX = 4.5
YMIN = -4.5
YMAX = 4.5
NX=50
NY=50

In [ ]:
grid_x, grid_y = np.mgrid[XMIN:XMAX:50j, YMIN:YMAX:50j]

In [ ]:
cmap="jet"

## Utility Functions

In [ ]:
def get_list_of_position(arr):
    """
    Return the list of positions
    """
    
    
    ListOfPositions = [] 
    arrsize=len(arr)
    Npos=0
    
    for idx in np.arange(arrsize):
    
        currentposition = (arr[idx][1], arr[idx][2] )
    
        if currentposition not in ListOfPositions:
            ListOfPositions.append(currentposition)
            Npos+=1
            
            
    return Npos, ListOfPositions
    

In [ ]:
def get_list_of_transmissions(arr):
    """
    
    """
    
    Narr=len(arr)
    Npos,list_of_position = get_list_of_position(arr)
    
    collectedtransmissions = np.empty(Npos, dtype=np.object)
    
    for idx in np.arange(Narr):   
        currentposition = (arr[idx][1], arr[idx][2])
        currentvalues = (arr[idx][3],arr[idx][4], arr[idx][5] )
  
    
        idx2=-1
        for position in list_of_position:
            idx2+=1
            
            if position == currentposition:
            
                if collectedtransmissions[idx2] == None:
                    collectedtransmissions[idx2] = []
                    collectedtransmissions[idx2].append(currentvalues)
                else:
                    collectedtransmissions[idx2].append(currentvalues)
                       
    
    return collectedtransmissions        

In [ ]:
def find_nearest_point(list_of_points,x0=0,y0=0):
    
    N=len(list_of_points)
    distances = []
    
    for pos in list_of_points:
        dx=pos[0]-x0
        dy=pos[1]-y0
        distances.append(np.sqrt(dx**2 + dy**2))
    
    distances = np.array(distances)
    
    idx0 = np.where(distances == distances.min())[0][0]
    
    return idx0,distances[idx0]
    

In [ ]:
def get_transmission_center(arr,x0=0,y0=0):
    """
    return the transmission of existing position (x0,y0) 
    
    """
    
    wl0 = np.array([], dtype=np.float64)
    eff0 = np.array([], dtype=np.float64)
    eeff0 = np.array([], dtype=np.float64)
    
    Npos,list_of_position = get_list_of_position(arr)
    arr_ext = get_list_of_transmissions(arr)
    
    idx0,d0=find_nearest_point(list_of_position,x0=x0,y0=y0)
     
    
    # loop on different positions
    for idx in np.arange(Npos):    
        list_of_datapoints = arr_ext[idx]
        x,y = list_of_position[idx]
    
        if idx==idx0:
            wl,eff,eeff = zip(*list_of_datapoints)   
            wl0 = wl
            eff0 = eff
            eeff0 =eeff          
        
    return wl0,eff0,eeff0
       

In [ ]:
def get_gridded_interpolated1d_effratio(wlin,arr,x0=0,y0=0):
    """
    Return the grid of interpolated efficiencies ratio wrt point position (x0,y0)
    
    input:
    - wlin : input wavelength
    - input array
    
    output:
    - list of efficiencies ratio related to the list_of_position
    
    """
    # ratio taken with respect to (x0,y0)
    wl0,eff0,eeff0 = get_transmission_center(arr,x0=x0,y0=y0)
    f0 = interpolate.interp1d(wl0,eff0)
    eff_0 = f0(wlin)
    
    
    Npos,list_of_position = get_list_of_position(arr)
    arr_ext = get_list_of_transmissions(arr)
    
    all_effratio=np.zeros(Npos)
    
    # loop on different positions
    for idx in np.arange(Npos):    
        list_of_datapoints = arr_ext[idx]
        x,y = list_of_position[idx]
        wl,eff,eeff = zip(*list_of_datapoints)   
        
        #print("get_gridded_interpolated1d_eff : ",min(wl),max(wl))
           
        f = interpolate.interp1d(wl,eff)
        
        eff = f(wlin)
        effratio = eff/eff_0
        all_effratio[idx] = effratio
        
    return all_effratio    

In [ ]:
def get_interpolated2d_effratio(arr,interpolation_method='rbf'):
    """
    Cube of transmission efficiency ratio
    
    """  
    Npos,list_of_position = get_list_of_position(arr)
    
    arr_ext = get_list_of_transmissions(arr)
    
    x,y = list(zip(*list_of_position))
    
    
    # container of interpolated efficiencies ratio
    all_interp_effratio_data =np.zeros((NX,NY,NWL))
    
    # loop on wavelength
    for idxwl,thewl in np.ndenumerate(WL):
        indexwl = idxwl[0]
        
        
        # list of efficiencies ratio at that wavelength of each position
        effratio = get_gridded_interpolated1d_effratio(thewl,arr)
        
        if interpolation_method == 'rbf':
            rbf = Rbf(x, y, effratio)
            Z = rbf(grid_x, grid_y)
            
        all_interp_effratio_data[:,:,indexwl] = Z.T
        
    return all_interp_effratio_data
       

# Read Input file

In [ ]:
datadir="data"
files_list= os.listdir(datadir)
print(files_list)
filename=files_list[1]
filename = '20200211-holo-4-003-uniformity-datacube.npy'
fullfilename=os.path.join(datadir,filename)
print(fullfilename)

In [ ]:
arr=np.load(fullfilename)

In [ ]:
arr.shape

In [ ]:
NARR=arr.shape[0]

In [ ]:
arr

In [ ]:
order0 = arr["order"]== 0
order1 = arr["order"]== 1
order2 = arr["order"]== 2

In [ ]:
arr_0 = arr[order0]
arr_1 = arr[order1]
arr_2 = arr[order2]

# Order 1

## List of positions

In [ ]:
Npos1,list_of_position_1 = get_list_of_position(arr_1)

In [ ]:
Npos1

In [ ]:
print(list_of_position_1)

In [ ]:
#for idx in range(Npos1):
#    print(idx,list_of_position_1[idx])

In [ ]:
#find_nearest_point(list_of_position_1,x0=4,y0=-4)

In [ ]:
arr_1_ext = get_list_of_transmissions(arr_1)

In [ ]:
arr_1_ext

## Make Multiarray

In [ ]:
effratio_multiarray = get_interpolated2d_effratio(arr_1)

In [ ]:
effratio_multiarray.shape

In [ ]:
effratio_multiarray.min()

In [ ]:
effratio_multiarray.max()

In [ ]:
vmin=0.85
vmax=1.15
bounds = (XMIN,  XMAX,YMIN, YMAX)

In [ ]:
idx=0
def producefig(idx,figdir="./",figtootname="uniftransmratio",numsize=3,figtype="png"):
    
    figname = figtootname+"_"+str(idx).zfill(numsize)+"."+ figtype
    fullfigname=os.path.join(figdir,figname)
    
    fig, ax = plt.subplots(nrows=1, ncols=1,figsize=(8,8))
    im=ax.imshow(effratio_multiarray[:,:,idx],origin="lower",cmap='jet',vmin=vmin,vmax=vmax,extent=bounds)
    divider = make_axes_locatable(ax)
    ax_divider = make_axes_locatable(ax)
    # Add an axes to the right of the main axes.
    cax = ax_divider.append_axes("right", size="4%", pad="2%")
    cb = fig.colorbar(im, cax=cax)
    cb.set_label('transmission ratio', rotation=90)
    cb.ax.get_yaxis().labelpad = 15
    ax.set_xlabel("X(mm)")
    ax.set_ylabel("Y(mm)")
    thewl=WL[idx]
    thetitle=f"Uniformity of transmission ratio $\lambda$ = {thewl} nm"
    ax.set_title(thetitle,fontsize=16,fontweight="bold")
    
    circle = plt.Circle((0, 0), 1, color='r',fill=False)
    ax.add_patch(circle)
    
    plt.savefig(fullfigname)
    #plt.show()


In [ ]:
for iwl in range(NWL):
    producefig(iwl)

# Convert in animated gif with image magick

In [ ]:
# convert -delay 1 -loop 0 *.png animation.gif

In [ ]:
import subprocess
def grid2gif(image_str, output_gif):
    str1 = 'convert -delay 1 -loop 0 ' + image_str  + ' ' + output_gif
    subprocess.call(str1, shell=True)

In [ ]:
grid2gif("uniftransmratio_*.png","multiuniftransmratio.gif")